# Analysis and Processing of Medicine Quantity Data from Excel Files

## I. Libraries import and file configurations

In [65]:
import pandas as pd # for data manipulation
import re # for regular expression operations

In [66]:
# The name of the excel file to be read
file_name = 'test.xlsx'

# Specify the columns to be read:
# - Column 10: Treatment method (phương pháp điều trị)
# - Column 11: Name of the doctor (tên bác sĩ)
require_cols = [10, 11]

# The row number of the header
# Every rows before this row will be ignored
header_row = 6

## II. Data processing

### Step 1: Load and clean data

In [67]:
# Read data from the file
df = pd.read_excel(file_name, usecols=require_cols, header=header_row)

# Drop rows with any missing values
df = df.dropna()

print('Data loaded and cleaned successfully!\n')
print(df.head())

Data loaded and cleaned successfully!

                                Phương pháp điều trị     Y, BS khám bệnh
1  SPORAL  100mg SL: 1 SN: 7; LINEZIN  4mg/ml SL:...  TRỊNH THỊ MINH HẢI
2  SPORAL  100mg SL: 14 SN: 7; LINEZIN  4mg/ml SL...  TRỊNH THỊ MINH HẢI
3  POVIDINE   10%, 20ml SL: 1 SN: ; COTRIMOXAZOLE...         HOÀNG LƯƠNG
4  CIPROBAY 500mg SL: 14 SN: 7; ACEMUC  200mg SL:...         HOÀNG LƯƠNG
5  METRONIDAZOL  250mg SL: 14 SN: 7; AUGMENTIN  1...         HOÀNG LƯƠNG


### Step 2: Extract doctor names

In [68]:
# Extract doctor names from the last column of the DataFrame
doctor_names = df.iloc[:, -1]

# Remove duplicate names
unique_doctor_names = doctor_names.drop_duplicates()

# Convert the unique doctor names to a list
doctor_names_list = unique_doctor_names.tolist()

print('Doctor names extracted successfully!\n')

Doctor names extracted successfully!



### Step 3: Process medicine names and quantities for each doctor

In [69]:
# Create a dictionary to store the medicine names and quantities
medicine_dict = {}

# Function to parse the medicine name and quantity from a string
def parse_medicine(medicine_str):
    # Use regular expression to extract the medicine name and its quantity
    # E.g.: 'SPORAL  100mg SL: 1 SN: 7' -> name: 'SPORAL  100mg', quantity: '1'
    match = re.match(r'(.+?) SL: (\d+)', medicine_str)

    # If the regular expression matches, return the medicine name and quantity
    if match:
        return match.groups()

    # If the regular expression does not match, return 'None' for both
    return None, None

# Iterate through each row in the DataFrame
# to process the medicine names and quantities for each doctor
for index, row in df.iterrows():
    # Get the doctor name from the last column
    doctor_name = row.iloc[-1]

    # Get the medicines from the second-to-last column and split them by ';'
    medicines = row.iloc[0].split(';')
    
    # Iterate through each medicine in the list of medicines
    for medicine in medicines:
        med_name, quantity = parse_medicine(medicine.strip())
        
        if med_name and quantity:
            # If the medicine name is not in the dictionary,
            # add it to the dictionary with the doctor names as keys and the quantities as values
            if med_name not in medicine_dict:
                medicine_dict[med_name] = {doctor: 0 for doctor in doctor_names_list}

            # Add the quantity to the corresponding doctor's value in the dictionary
            medicine_dict[med_name][doctor_name] += int(quantity)

print('Medicine names and quantities processed successfully!\n')

# Print some values of the medicine_dict to the console
print('Here are the first 3 medicine names and its quantities prescribed by each doctor:')
for med_name, quantities in list(medicine_dict.items())[:3]:
    print(f"Medicine: {med_name}")

    for doctor, quantity in quantities.items():
        print(f"  - {doctor}: {quantity}")
print('\n')

Medicine names and quantities processed successfully!

Here are the first 3 medicine names and its quantities prescribed by each doctor:
Medicine: SPORAL  100mg
  - TRỊNH THỊ MINH HẢI: 15
  - HOÀNG LƯƠNG: 0
  - TRƯƠNG THẾ ANH: 0
  - TRƯƠNG KIM TRI: 0
Medicine: LINEZIN  4mg/ml
  - TRỊNH THỊ MINH HẢI: 42
  - HOÀNG LƯƠNG: 0
  - TRƯƠNG THẾ ANH: 0
  - TRƯƠNG KIM TRI: 0
Medicine: ENTEROGERMINA  4 tỷ/5ml
  - TRỊNH THỊ MINH HẢI: 28
  - HOÀNG LƯƠNG: 0
  - TRƯƠNG THẾ ANH: 0
  - TRƯƠNG KIM TRI: 0




### Step 4: Calculate the total quantity of each medicine

In [70]:
# Create a DataFrame from the medicine dictionary
# with doctor names as columns headers and medicine names as index labels
result_df = pd.DataFrame.from_dict(medicine_dict, orient='index', columns=doctor_names_list)

# Add a new column 'Total' that contains the sum of all quantities in each row
result_df['>>> Total <<<'] = result_df.sum(axis=1)

print('Sum of each medicine quantity calculated successfully!\n')

Sum of each medicine quantity calculated successfully!



### Step 5: Write the processed data to a new sheet

In [71]:
print('Writing results...\n')

# Write the DataFrame to a new sheet called 'Result'
with pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    result_df.to_excel(writer, sheet_name='Result', startrow=0)

print('>>> ALL DONE! <<<')
print('Medicine names and quantities of each doctor written to the "Result" sheet.')

Writing results...

>>> ALL DONE! <<<
Medicine names and quantities of each doctor written to the "Result" sheet.
